In [1]:
import os

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbbai-azure-ai-agentic-frameworks"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")


Directory changed to C:\Users\pablosal\Desktop\gbbai-azure-ai-agentic-frameworks


## Extending ConversableAgent: A Study in Autogen Extensibility

The preceding discussion illuminated the principle of software extensibility through the lens of the ConversableAgent framework. At its core, software extensibility is the capacity of a system to incorporate additional features with minimal alterations to the established codebase. This attribute is indispensable for the sustainable growth and scalability of software architectures.

In [10]:
ConversableAgent?

Init signature:
ConversableAgent(
    name: str,
    system_message: Union[str, List, NoneType] = 'You are a helpful AI Assistant.',
    is_termination_msg: Optional[Callable[[Dict], bool]] = None,
    max_consecutive_auto_reply: Optional[int] = None,
    human_input_mode: Literal['ALWAYS', 'NEVER', 'TERMINATE'] = 'TERMINATE',
    function_map: Optional[Dict[str, Callable]] = None,
    code_execution_config: Union[Dict, Literal[False]] = False,
    llm_config: Union[Dict, Literal[False], NoneType] = None,
    default_auto_reply: Union[str, Dict] = '',
    description: Optional[str] = None,
    chat_messages: Optional[Dict[autogen.agentchat.agent.Agent, List[Dict]]] = None,
)
Docstring:     
(In preview) A class for generic conversable agents which can be configured as assistant or user proxy.

After receiving each message, the agent will send a reply to the sender unless the msg is a termination msg.
For example, AssistantAgent and UserProxyAgent are subclasses of this class,
configure

In [9]:
from autogen import ConversableAgent, AssistantAgent
from datetime import datetime
import yaml
from typing import Dict, List, Optional, Union, Callable, Literal, Any
import os
from utils.ml_logging import get_logger
from datetime import datetime

def get_llm_config(
    azure_openai_key: Optional[str] = None,
    azure_aoai_chat_model_name_deployment_id: Optional[str] = None,
    azure_openai_api_endpoint: Optional[str] = None,
    azure_openai_api_version: Optional[str] = None
) -> Dict[str, List[Dict[str, str]]]:
    """
    Generate a configuration list dictionary for the LLM from provided parameters or environment variables.

    Args:
        azure_openai_key (Optional[str]): The Azure OpenAI key.
        azure_aoai_chat_model_name_deployment_id (Optional[str]): The Azure AOAI chat model name deployment ID.
        azure_openai_api_endpoint (Optional[str]): The Azure OpenAI API endpoint.
        azure_openai_api_version (Optional[str]): The Azure OpenAI API version.

    Returns:
        Dict[str, List[Dict[str, str]]]: A dictionary containing the configuration list.
    """
    azure_openai_key = azure_openai_key or os.getenv("AZURE_OPENAI_KEY")
    azure_aoai_chat_model_name_deployment_id = azure_aoai_chat_model_name_deployment_id or os.getenv("AZURE_AOAI_CHAT_MODEL_NAME_DEPLOYMENT_ID")
    azure_openai_api_endpoint = azure_openai_api_endpoint or os.getenv("AZURE_OPENAI_API_ENDPOINT")
    azure_openai_api_version = azure_openai_api_version or os.getenv("AZURE_OPENAI_API_VERSION")

    return {
        "config_list": [
            {
                "model": azure_aoai_chat_model_name_deployment_id,
                "api_type": "azure",
                "api_key": azure_openai_key,
                "base_url": azure_openai_api_endpoint,
                "api_version": azure_openai_api_version
            }
        ]
    }

class SuperConversableAgent(ConversableAgent):
    def __init__(
        self,
        agent_from_yaml: Optional[str] = None,
        name: Optional[str] = None,
        system_message: Optional[Union[str, List]] = "You are a helpful AI Assistant.",
        is_termination_msg: Optional[Callable[[Dict], bool]] = None,
        max_consecutive_auto_reply: Optional[int] = None,
        human_input_mode: Literal["ALWAYS", "NEVER", "TERMINATE"] = "TERMINATE",
        function_map: Optional[Dict[str, Callable]] = None,
        code_execution_config: Union[Dict, Literal[False]] = False,
        llm_config: Optional[Union[Dict, Literal[False]]] = None,
        default_auto_reply: Union[str, Dict] = "",
        description: Optional[str] = None,
        chat_messages: Optional[Dict[ConversableAgent, List[Dict]]] = None,
        avatar: str = '🤖',
        enable_logs: bool = True,
        log_dir: Optional[str] = None,
        verbose: bool = False
    ):
        """
        Initialize the SuperConversableAgent with LLM configuration.

        Args:
            name (str): Name of the agent.
                Example: "AssistantBot"
            
            system_message (str or list): System message for the ChatCompletion inference.
                Example: "You are a helpful AI Assistant."
            
            is_termination_msg (function): A function that takes a message in the form of a dictionary
                and returns a boolean value indicating if this received message is a termination message.
                The dict can contain the following keys: "content", "role", "name", "function_call".
                Example:
                    def is_termination(message: Dict) -> bool:
                        return message.get("content") == "exit"
            
            max_consecutive_auto_reply (int): The maximum number of consecutive auto replies.
                Defaults to None (no limit provided, class attribute MAX_CONSECUTIVE_AUTO_REPLY will be used as the limit in this case).
                When set to 0, no auto reply will be generated.
                Example: 5
            
            human_input_mode (str): Whether to ask for human inputs every time a message is received.
                Possible values are "ALWAYS", "TERMINATE", "NEVER".
                (1) When "ALWAYS", the agent prompts for human input every time a message is received.
                    Under this mode, the conversation stops when the human input is "exit",
                    or when is_termination_msg is True and there is no human input.
                (2) When "TERMINATE", the agent only prompts for human input only when a termination message is received or
                    the number of auto reply reaches the max_consecutive_auto_reply.
                (3) When "NEVER", the agent will never prompt for human input. Under this mode, the conversation stops
                    when the number of auto reply reaches the max_consecutive_auto_reply or when is_termination_msg is True.
                Example: "TERMINATE"
            
            function_map (dict[str, callable]): Mapping function names (passed to openai) to callable functions, also used for tool calls.
                Example: {"get_weather": get_weather_function}
            
            code_execution_config (dict or False): Config for the code execution.
                To disable code execution, set to False. Otherwise, set to a dictionary with the following keys:
                - work_dir (Optional, str): The working directory for the code execution.
                    If None, a default working directory will be used.
                    The default working directory is the "extensions" directory under "path_to_autogen".
                - use_docker (Optional, list, str or bool): The docker image to use for code execution.
                    Default is True, which means the code will be executed in a docker container. A default list of images will be used.
                    If a list or a str of image name(s) is provided, the code will be executed in a docker container
                    with the first image successfully pulled.
                    If False, the code will be executed in the current environment.
                    We strongly recommend using docker for code execution.
                - timeout (Optional, int): The maximum execution time in seconds.
                - last_n_messages (Experimental, int or str): The number of messages to look back for code execution.
                    If set to 'auto', it will scan backwards through all messages arriving since the agent last spoke, which is typically the last time execution was attempted. (Default: auto)
                Example:
                    {
                        "work_dir": "/path/to/work_dir",
                        "use_docker": True,
                        "timeout": 60,
                        "last_n_messages": "auto"
                    }
            
            llm_config (dict or False or None): LLM inference configuration.
                Please refer to [OpenAIWrapper.create](/docs/reference/oai/client#create) for available options.
                When using OpenAI or Azure OpenAI endpoints, please specify a non-empty 'model' either in `llm_config` or in each config of 'config_list' in `llm_config`.
                To disable llm-based auto reply, set to False.
                When set to None, will use self.DEFAULT_CONFIG, which defaults to False.
                Example:
                    {
                        "model": "gpt-3.5-turbo",
                        "temperature": 0.7
                    }
            
            default_auto_reply (str or dict): Default auto reply when no code execution or llm-based reply is generated.
                Example: "I'm not sure how to help with that."
            
            description (str): A short description of the agent. This description is used by other agents
                (e.g. the GroupChatManager) to decide when to call upon this agent. (Default: system_message)
                Example: "This agent assists with weather-related queries."
            
            chat_messages (dict or None): The previous chat messages that this agent had in the past with other agents.
                Can be used to give the agent a memory by providing the chat history. This will allow the agent to
                resume previous conversations. Defaults to an empty chat history.
                Example:
                    {
                        other_agent: [
                            {"role": "user", "content": "Hello!"},
                            {"role": "assistant", "content": "Hi there!"}
                        ]
                    }
            
            avatar (str): The avatar for the agent. Defaults to a robot emoji.
                Example: "🤖"
            
            logging (bool): Whether to log the agent's activities. Defaults to True.
                Example: True
            
            verbose (bool): Whether the agent should process messages silently. Defaults to False.
                Example: False
        """

        if agent_from_yaml:
            config = self.load_agent_from_yaml(agent_from_yaml)
            llm_config = self._resolve_llm_config(config, llm_config)
            super().__init__(
                name=config.get('name', 'DefaultAgentName'),
                system_message=config.get('system_message', ["You are a helpful AI Assistant."]),
                is_termination_msg=config.get('is_termination_msg', None),
                max_consecutive_auto_reply=config.get('max_consecutive_auto_reply', 1),
                human_input_mode=config.get('human_input_mode', "TERMINATE"),
                function_map=config.get('function_map', {}),
                code_execution_config=config.get('code_execution_config', False),
                llm_config=llm_config,
                default_auto_reply=config.get('default_auto_reply', ""),
                description=config.get('description', ""),
                chat_messages=config.get('chat_messages', {}),
            )
            self.avatar = config.get('avatar', '🤖')
            self.verbose = config.get('verbose', False)
            self.log_dir = config.get('log_dir', None)
            self.enable_logs = config.get('enable_logs', True)
            if self.enable_logs:
                agent_name = config.get('name', 'DefaultAgentName')
                current_date = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
                log_file_name = f"{agent_name}_{current_date}.log"
                log_file_path = os.path.join(self.log_dir, log_file_name) if self.log_dir else log_file_name
                self.logger = get_logger(log_file=log_file_path)
            else:
                self.logger = get_logger()
        else:
            self.avatar = avatar
            self.verbose = verbose
            self.enable_logs = enable_logs
            self.log_dir = log_dir
            if self.enable_logs:
                agent_name = name if name else "DefaultAgentName"
                current_date = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
                log_file_name = f"{agent_name}_{current_date}.log"
                log_file_path = os.path.join(self.log_dir, log_file_name) if self.log_dir else log_file_name
                self.logger = get_logger(log_file=log_file_path)
            else:
                self.logger = get_logger()

            super().__init__(
                name=name,
                system_message=system_message,
                is_termination_msg=is_termination_msg,
                max_consecutive_auto_reply=max_consecutive_auto_reply,
                human_input_mode=human_input_mode,
                function_map=function_map,
                code_execution_config=code_execution_config,
                llm_config=llm_config,
                default_auto_reply=default_auto_reply,
                description=description,
                chat_messages=chat_messages,
            )

        # Log the agent being loaded with all the information
        self.logger.info("Agent loaded with the following configuration:")

        # Dynamically log all attributes of the client
        for attribute, value in self.__dict__.items():
            formatted_value = f"\n{value}" if isinstance(value, dict) or isinstance(value, list) else value
            self.logger.info(f"    {attribute}: {formatted_value}")

    @classmethod
    def load_agent_from_yaml(cls, yaml_file: str, llm_config_dict: Optional[Dict[str, Any]] = None) -> 'SuperConversableAgent':
        """
        Load an agent configuration from a YAML file.

        Args:
            yaml_file (str): Path to the YAML configuration file.
            llm_config_dict (Optional[Dict[str, Any]]): Custom LLM configuration dictionary.

        Returns:
            SuperConversableAgent: An instance of SuperConversableAgent.
        """
        with open(yaml_file, 'r') as file:
            config = yaml.safe_load(file)
    
        return config
    
    # def receive(self, message: Dict[str, Any], sender: 'ConversableAgent', request_reply: bool = True, silent: bool = False):
    #     """
    #     Receives a message, logs the interaction, and sends data to a remote database.
    #     """
    #     try:
    #         super().receive(message, sender, request_reply, silent)
    #         timestamp = datetime.now().isoformat()
    #         log_entry = {
    #             "timestamp": timestamp,
    #             "action": "receive",
    #             "sender": sender.name,
    #             "receiver": self.name,
    #             "message": message
    #         }
    #         if self.enable_logs:
    #             self.logger.info(log_entry)
            
    #     except Exception as e:
    #         self.logger.error(f"Error processing message: {e}")

    # def send(self, message: Dict[str, Any], recipient: 'ConversableAgent'):
    #     """
    #     Sends a message to a recipient, logs the interaction, and sends data to a remote database.
    #     """
    #     try:
    #         super().send(message, recipient)
    #         timestamp = datetime.now().isoformat()
    #         log_entry = {
    #             "timestamp": timestamp,
    #             "action": "send",
    #             "sender": self.name,
    #             "receiver": recipient.name,
    #             "message": message
    #         }
    #         if self.logging:
    #             self.logger.info(log_entry)
        
    #     except Exception as e:
    #         self.logger.error(f"Error sending message: {e}")

    def _process_received_message(self, message, sender, silent):
        """
        Process received message and log it.

        Args:
            message (str): The message content.
            sender (ConversableAgent): The sender of the message.
            silent (bool, optional): Whether the message should be processed silently. Defaults to the instance's silent attribute.
        """
        try:
            timestamp = datetime.now().isoformat()
            log_entry = {
                "timestamp": timestamp,
                "action": "receive",
                "sender": sender.name,
                "receiver": self.name,
                "message": message
            }
            if self.logging:
                self.logger.info(log_entry)

            if self.verbose:
                self._display_message(sender, message)

            return super()._process_received_message(message, sender, silent)
        except Exception as e:
            self.logger.error(f"Error processing received message: {e}")
    
    def _display_message(self, sender, message):
        """
        Helper function to display a message in the chat interface.

        Args:
            sender (ConversableAgent): The sender of the message.
            message (str): The message content.
        """
        # Text-based formatting for the message
        formatted_message = f"Sender: {sender.name}\nMessage: {message}\n"

        # Print the formatted message to the console
        print(formatted_message)

        # Mapping agent names to emojis
        agent_name_to_emoji = {
            "User": "👤",
            "MedicalResearchPlanner": "🧑🏿‍💼",
            "FinalMedicalReviewer": "👨🏽‍⚕️",
            "MedicalResearcher": "👩‍⚕️"
        }

        # Determine the avatar based on the sender's name
        if sender.name in agent_name_to_emoji:
            avatar = agent_name_to_emoji[sender.name]
        else:
            avatar = "❓"

        # Print the avatar and formatted message
        print(f"{avatar} {formatted_message}")
    
    @staticmethod
    def _resolve_llm_config(config: Dict[str, Any], llm_config_dict: Optional[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Resolve the LLM configuration based on the provided configuration and custom dictionary.

        Args:
            config (Dict[str, Any]): Configuration dictionary from the YAML file.
            llm_config_dict (Dict[str, Any], optional): Custom LLM configuration dictionary.

        Returns:
            Dict[str, Any]: Resolved LLM configuration.
        """
        if config.get('llm_config') == "default":
            return get_llm_config()
        elif config.get('llm_config') == "custom":
            return llm_config_dict if llm_config_dict else {}
        return config.get('llm_config', {})



In [24]:
# researcher_agent = SuperConversableAgent(agent_from_yaml=r"src\app\agents\MedicalResearcher.yaml")
# planner_agent = SuperConversableAgent(agent_from_yaml=r"src\app\agents\MedicalResearchPlanner.yaml")
# reviewer_agent = SuperConversableAgent(agent_from_yaml=r"src\app\agents\MedicalReviewer.yaml")

In [3]:
llm_config = get_llm_config()

medical_research_planner = SuperConversableAgent(
    name="MedicalResearchPlanner",
    system_message=(
        "Given a research task, your role is to determine the specific information needed to comprehensively support the research. "
        "You will assess the task's progress and delegate sub-tasks to other agents as needed. If no more information is needed and document looks good, mention STOP or TERMINATE"
    ),
    llm_config=llm_config,
    avatar="📝"
)

reviewer_agent = SuperConversableAgent(
    name="FinalMedicalReviewer",
    system_message=(
        "You are the final medical reviewer, tasked with aggregating and reviewing feedback from other reviewers. "
        "Your role is to make the final decision on the content's readiness for publication, ensuring it adheres to all legal, "
        "security, and ethical standards. If documentation is ready for public circulation, mention STOP or TERMINATE"
    ),
    llm_config=llm_config,
    avatar="🔍"
)

researcher_agent = SuperConversableAgent(
    name="MedicalResearcher",
    system_message=(
        "As a Medical Researcher, your role is to draft a comprehensive manuscript detailing your study's findings. "
        "Ensure the manuscript is scientifically robust, covering all critical aspects of your research."
    ),
    llm_config=llm_config,
    avatar="🔬"
)

In [4]:
from autogen import GroupChat, GroupChatManager
import autogen

agents_dict = {
    "FinalMedicalReviewer": reviewer_agent,
    "MedicalResearcher": researcher_agent,
}


# Define the group chat for the medical use case with limited agents
medical_groupchat = GroupChat(
    agents=[
        agents_dict["FinalMedicalReviewer"], 
        agents_dict["MedicalResearcher"]
    ],
    messages=[],
    max_round=3,
    allowed_or_disallowed_speaker_transitions={
        agents_dict["FinalMedicalReviewer"]: [
            agents_dict["MedicalResearcher"]
        ],
        agents_dict["MedicalResearcher"]: [
            agents_dict["FinalMedicalReviewer"]
        ]
    },
    speaker_transitions_type="allowed",
)

# Initialize the GroupChatManager with the medical group chat and LLM configuration
medical_manager = GroupChatManager(
    groupchat=medical_groupchat, 
    llm_config=get_llm_config(),
    system_message=""" 
    - "Your role is to orchestrate the entire research process for a given task."
    - "Determine the specific information needed to comprehensively support the research."
    - "Assess the task's progress and delegate sub-tasks to other agents as needed."
    - "Collaborate with the MedicalResearcher to draft a comprehensive manuscript."
    - "Work with the MedicalReviewer to ensure the manuscript meets all legal, security, and ethical standards."
    - "Once all necessary information is gathered and the document is ready, clearly conclude the task by mentioning STOP or TERMINATE."
    """
)

In [6]:
# Define the task for the initial content generation
task = '''
        Create a comprehensive medical research document detailing the study on 
        the effects of a new drug on heart disease. Include methodology, results, 
        discussion, and conclusion sections. Ensure the document adheres to medical 
        research standards and ethical guidelines. Please support your findings with
        relevant references from PubMed.
       '''

import re
import uuid

terminate_pattern = re.compile(r".*STOP.*", re.IGNORECASE)
terminate_variation_pattern = re.compile(r".*TERMINATED.*", re.IGNORECASE)
finalize_pattern = re.compile(r".*TERMINATE.*", re.IGNORECASE)

is_termination_msg = lambda x: any([
    terminate_pattern.search(x.get("content", "")),
    terminate_variation_pattern.search(x.get("content", "")),
    finalize_pattern.search(x.get("content", "")),
])

# Generate a unique ID for the run
run_id = str(uuid.uuid4())
print("Run ID: " + run_id)

# Start logging with the run ID
logging_session_id = autogen.runtime_logging.start(logger_type="file", config={"filename": f"run_{run_id}.log"})
print("Logging session ID: " + str(logging_session_id))

chat_result = await researcher_agent.a_initiate_chat(
    recipient=medical_manager,
    message=task,
    max_turns=2,
    is_termination_msg=is_termination_msg
)
autogen.runtime_logging.stop()

# Print the final document result
print("Final Document:", chat_result.chat_history[-1]['content'])

Run ID: df4c1b1c-fc08-483a-af68-ecebe02e9856
Logging session ID: 00a02365-918d-4443-a57b-5331c1bbd3c2
MedicalResearcher (to chat_manager):


        Create a comprehensive medical research document detailing the study on 
        the effects of a new drug on heart disease. Include methodology, results, 
        discussion, and conclusion sections. Ensure the document adheres to medical 
        research standards and ethical guidelines. Please support your findings with
        relevant references from PubMed.
       

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


IndexError: list index out of range